In [1]:
import gurobipy as grb
import numpy as np
import itertools
import sys
import copy
import collections

In [26]:
def iterer(*args):
    return itertools.product(*[x_ if isinstance(x_,collections.Iterable) else range(x_) for x_ in args])

In [149]:
"""
    places in heaps: (1 is closer to orange side, 0 to central tracking device)
       | 1|
    | 0| 4| 2|
       | 3|

    colors of cubes: (same as for orange side)
    
    black  -- 0
    grean  -- 1
    blue   -- 2
    orange -- 3
    yellow -- 4
    
    numbers of heaps - 0,1,2,3,4,5 CCW starting from closest heap to central tracking device on the orange side
    
    man = 0, 1, 2 (picking 1 4 3 if from 0 for orange!)
"""
class STNode():
    def __init__(self, state, upper):
        self.state = state
        self.upper = upper
        self.nexts = []
        

class SimpleTree():
    def __init__(self, root)

class State():
    X = [0,1,2]
    def __init__(self, lines, heights = [0,0,0], x = 0, y = 0, time = 0):
        self.lines = lines
        self.heights = heights
        self.x = x
        self.y = y
        self.time = time
    
       
    def get_next_allowed_cube(self):
        a = 4
        for l in self.lines:
            for c in l:
                if c > 0:
                    a = min(c, a)
        # return 4 if whole plan picked
        return a
    
    def get_next_line(self):
        for x, l in zip(self.X, self.lines):
            any_cube = False
            for c in l:
                if c != -1:
                    any_cube = True
            if any_cube:
                return x,l
            
    def __str__(self):
        s = "---STATE---" + '\n'
        for l in self.lines:
            s += str(l) + '\n'
        s += "x=%d, y=%d, time=%f\n"%(self.x, self.y, self.time)
        s += "height = " + str(self.heights) + '\n'
        s += '-----------'
        return s
    
    def is_final(self):
        # if all is -1
        return sum(sum(lines)) == -9
    
class SuctionRobotStrategyOptimizer():
    defaults = {
        'plan': [0,1,2],
        'allowed_sides': [
            [False, True, True, True],
            [True, False, True, True],
            [True, True, False, True],
            [True, True, False, True],
            [True, True, False, True],
            [True, True, True, False]
        ],
        'heaps': [0,1,2],
        'pick_time': 1,
        'move_time': 1,
        'heap': {'orange':np.array([
                        [-1, 0,-1],
                        [ 1, 4, 3],
                        [-1, 2,-1]
                    ]),
                 'green':np.array([
                        [-1, 0,-1],
                        [ 3, 4, 1],
                        [-1, 2,-1]
                 ])
            }
    }
    def __init__(self, **kvargs):
        self.params = {}
        for k,v in self.defaults.items():
            self.params[k] = copy.copy(v)
            
        for k,v in kvargs.items():
            if k in self.params.keys():
                self.params[k] = copy.copy(v)
                
        self.manipulator_masks = [list(x) for x in itertools.product([0,1], repeat=3)][1:]
        
    def rotate_heap(self, heap, side):
        return np.rot90(heap, side)
    
    def get_heap(self, number = 0, side = 0):
        heap = None
        if number in [0,1,2]:
            heap = self.params['heap']['orange']
        if number in [4,5,6]:
            heap = self.params['heap']['green']
        return self.rotate_heap(heap,side)
    
    def set_plan(self, plan):
        self.params['plan'] = plan
        
    def get_plan(self, num = None):
        if num == None:
            return self.params['plan']
        elif num in [0,1,2]:
            return self.params['plan'][num]
        return self.params['plan']
    
    def reverse_plan(self):
        self.params['plan'] = self.params['plan'][::-1]
        
    def get_move_time(self, dx, dy):
        return max(abs(dx),abs(dy))*self.params['move_time']
    
    def get_lines_by_plan_and_heap(self, heap):
        lines = copy.copy(heap)
        for i, j in iterer(3,3):
            if heap[i][j] in range(5):
                if heap[i][j] in self.get_plan():
                    lines[i][j] = self.get_plan().index(heap[i][j]) + 1
                else:
                    lines[i][j] = 0
        return lines
    
    def try_to_pick(self, state, y, l, c, x, man, mans):
        picked = [-1,-1,-1]
        for i in range(3): # run over mans
            if mans[i] == 1: # if need to pick by i-th man
                if i + y in [0,1,2] and l[i+y] >= 0: # check if we pick existing cube
                    # print(i,y)
                    if (i == man and (c == l[i+y] or c == 1)) or (i != man and c != l[i+y]) : # if we can pick this cube according to plan 
                        picked[i] = l[i+y] # add to picked (why?)
                        state.heights[i] += 1 # increase height of i-th man
                        state.lines[x][i+y] = -1 # remove cube from heap
                        # print(i, y, i+y)
                    else:
                        return None
                else:
                    return None
        # if still not return, add time and return new state
        state.time += self.params['pick_time']
        state.time += self.get_move_time(x-state.x, y-state.y)
        state.x = x
        state.y = y
        return state
    
    def get_new_states(self, state, man):
        c = state.get_next_allowed_cube()
        x, l = state.get_next_line()
        
        for mans, y in iterer(self.manipulator_masks, range(-2,3)):
            new_state = self.try_to_pick(copy.deepcopy(state), y, l, c, x, man, mans)
            if new_state != None:
                # print(y, mans)
                yield new_state
                
        
        
    def get_sequence_1tower(self, heap_num, side, man):
        h = self.get_heap(heap_num, side)
        if h[1][-1] == self.get_plan(1):
            # not possible to pick
            return None 
        

In [150]:
opt = SuctionRobotStrategyOptimizer()

In [151]:
s = State(opt.get_lines_by_plan_and_heap(opt.get_heap()))

In [152]:
print(s)

---STATE---
[-1  1 -1]
[2 0 0]
[-1  3 -1]
x=0, y=0, time=0.000000
height = [0, 0, 0]
-----------


In [160]:
nexts = [ns for ns in opt.get_new_states(s, 0)]
next_one = None
for nxt in nexts:
    for ns in opt.get_new_states(nxt, 0):
        if ns.heights == [2,1,1]:
            next_one = ns
    

In [161]:
print(next_one)

---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1  3 -1]
x=1, y=0, time=4.000000
height = [2, 1, 1]
-----------


In [162]:
for ns in opt.get_new_states(next_one, 0):
    print(ns)

---STATE---
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
x=2, y=1, time=6.000000
height = [3, 1, 1]
-----------


In [136]:
c = s.get_next_allowed_cube()
x, l = s.get_next_line()
mans = [0,1,0]
y = 0
man = 0
new_state = opt.try_to_pick(copy.deepcopy(s), y, l, c, x, man, mans)
print(new_state)

1 0
1 0 1
---STATE---
[-1 -1 -1]
[ 2 -1  0]
[-1  3 -1]
x=1, y=0, time=2.000000
height = [0, 3, 1]
-----------


In [166]:
sum(sum(s.lines))

2

In [167]:
s.lines

array([[-1,  1, -1],
       [ 2,  0,  0],
       [-1,  3, -1]])